# Using external minimizers

We show how to use an external minimizer to find the minimum of a function and then use iminuit to compute the parameter uncertainties.

We will demonstrate this with a maximum-likelihood fit of a normal distribution, which is carried out with `scipy.optimize.minimize`. iminuit is then used to compute the parameter uncertainties.

Note: iminuit can call the scipy minimizers directly with `Minuit.scipy`, so `scipy.optimize.minimize` is only used here to demonstrate the general approach.

In [ ]:
from iminuit import Minuit
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

In [ ]:
# normally distributed data
rng = np.random.default_rng(1)
x = rng.normal(size=1000)

# negative log-likelihood for a normal distribution
def nll(par):
    return -np.sum(norm.logpdf(x, par[0], par[1]))

nll.errordef = Minuit.LIKELIHOOD

# minimize nll with scipy.optimize.minimize
result = minimize(nll, np.ones(2))
result

In [ ]:
# initialize Minuit with the fit result from scipy.optimize.minimize
m = Minuit(nll, result.x)
m.hesse()  # this also works without calling MIGRAD before

We can also compute the "Hesse errors" at any other point than the minimum. These cannot be interpreted as parameter uncertainties, they are just some numbers related to the second derivative of the cost function at that point.

In [ ]:
m.values = (1.0, 0.5)
m.hesse()

Minuit now reports that the minimum is invalid, which is correct, but it does not matter for the Hesse errors, which are computed anyway.

Likewise, it one can also run MINOS to get MINOS estimates. Note that MINOS can fail if the starting point is not actually a minimum. So here we reset the values to the solution found by scipy.optimize.

In [ ]:
m.values = result.x
m.minos()

We can see that MINOS ran successfully. The Hesse Errors were also updated, because MINOS needs HESSE to run first. HESSE is called automatically in this case.